## Лабораторная работа №7 Классификация тональности
#### Предмет: Методы интеллектуального анализа текстов
#### Студент: Леонова Алина, СБ № 1032212306, НФИмд-01-21

### 7.4. Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации

Данные для обучения моделей:

In [1]:
! wget https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv

--2022-10-23 13:39:44--  https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv [following]
--2022-10-23 13:39:45--  https://www.dropbox.com/s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc372cbf8fd296c44c23754ef4f5.dl.dropboxusercontent.com/cd/0/inline/BvXo-Gjzy4x34PWp4XKSno-RYIAi-Bdz-HuOkswVJsWh8cfkLypzKnH-CposQ-XLUqYv6lTJwoL4KJqoay6G4Lxg9UriA2P1rM1QZRVfCqC7FjAh9jtMiw3ONkEcQNRqnwu8bny8nOg9MC1vJ_6Y3UT0ILzRiFmP_8p4lof4_-4vNw/file# [following]
--2022-10-23 13:39:45--  https://uc372cbf8fd296c44c23754ef4f5.dl.dropboxusercontent.com/cd/0/inline/BvXo-Gjzy4x34PWp4XKSno-RYIAi-Bdz

Данные для тестирования моделей: 

In [2]:
! wget https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv

--2022-10-23 13:39:46--  https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv [following]
--2022-10-23 13:39:46--  https://www.dropbox.com/s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc496f1dac67180a4cdfbf633b08.dl.dropboxusercontent.com/cd/0/inline/BvWSCErAfVPtK5FFVqIJSHnSXQ5j3z7WtH64CsVT1c6kggeFS2XHGXSB6wutnyq0WTm4mNETrbro_JbCL7rLFp8vrqrqDbVhQRgCXPRKPjn_ZGlKgFjRwXuRNnIKYulUMBLqtIpA0K0-RJ9am0ZoCRkXlrRVxhdbX5s3-pLhUtBg8Q/file# [following]
--2022-10-23 13:39:46--  https://uc496f1dac67180a4cdfbf633b08.dl.dropboxusercontent.com/cd/0/inline/BvWSCErAfVPtK5FFVqIJSHnSXQ5j3z7WtH64CsVT1c6kggeFS2XHGXSB

В заданиях 7.1 - 7.3 заготовка кода есть в презентации.

In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv("rusentiment_random_posts.csv")
train = train.fillna(method="ffill")

test = pd.read_csv("rusentiment_test.csv")
test = test.fillna(method="ffill")

In [4]:
train

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...
...,...,...
21263,neutral,"Анастасия, у меня есть друг, с которым вы хоро..."
21264,neutral,Колька пошли гулять!!?
21265,neutral,Ура! Золото дают бесплатно!Напиши это в 4 комм...
21266,speech,"С Праздником, Ксюнь! \nЖенского счастья тебе!\..."


In [5]:
test

,label,text
0,neutral,"Александр, тебе к лицу эта пушка :)\n"
1,positive,"Скоро ты вернешься домой, грязный, не бритый н..."
2,neutral,помниш...))
3,skip,Мы просто с [id12210212|Елисеем] психанули )...
4,speech,"ой, прости забыла поздравить, с праздничьком, ..."
...,...,...
2962,neutral,Коллективно жмём F13! И наблюдаем за результатом!
2963,neutral,ану ка быстро цоя добавь!
2964,skip,Ты одна половинка - я вторая половинка...... А...
2965,positive,Люблю кису)))


#### Предобработка данных

In [6]:
# Очистка данных
import nltk
import string, re
from collections import Counter, OrderedDict
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\W+', gaps=True)
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')

# минимальная обработка текста
def clean(text):
    text = re.sub('ё','е',str(text))
    text = re.sub('\\n',' ',str(text)) # удаляет переносы строк
    text = re.sub('\s{2,}', " ", text) # удаляет лишние пробелы
    return text

# чистит от стоп слов токенезированный текст
def clean_from_stopwords(text):
    text = [token for token in text if re.match(r'[а-яА-Я]', token) and token not in stopwords]
    #text = re.sub('ё','е',str(text))
    text = re.sub(r'[^\яа-яА-Я0-9@]',' ',str(text))
    text = re.sub(r'\w*\d+\w*', '', text) # удаляет цифры
    text = re.sub('\s{2,}', " ", text) # удаляет лишние пробелы
    return text


train['text'] = train['text'].apply(clean)
test['text'] = test['text'].apply(clean)

# токенизирование
train['text_tokenized'] = train.text.map(lambda row: tokenizer.tokenize(row.lower()))
test['text_tokenized'] = test.text.map(lambda row: tokenizer.tokenize(row.lower()))

train['cleaned_text'] = train['text_tokenized'].apply(clean_from_stopwords)
test['cleaned_text'] = test['text_tokenized'].apply(clean_from_stopwords)

train

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,text,text_tokenized,cleaned_text
0,negative,"А попа подозревала давно,что ты с кавказа..пер...","[а, попа, подозревала, давно, что, ты, с, кавк...",попа подозревала давно кавказа перестану обща...
1,speech,З прошедшим Днем Ангела)))))))),"[з, прошедшим, днем, ангела]",з прошедшим днем ангела
2,skip,Два дня до отлета с острова!!!!!!!,"[два, дня, до, отлета, с, острова]",дня отлета острова
3,negative,"Блин, почему эта жизнь столь не справедлива ((((","[блин, почему, эта, жизнь, столь, не, справедл...",блин почему эта жизнь столь справедлива
4,skip,где еще встречать свой день рождения как не на...,"[где, еще, встречать, свой, день, рождения, ка...",встречать свой день рождения кладбище
...,...,...,...,...
21263,neutral,"Анастасия, у меня есть друг, с которым вы хоро...","[анастасия, у, меня, есть, друг, с, которым, в...",анастасия друг которым смотрелись вместе узнай
21264,neutral,Колька пошли гулять!!?,"[колька, пошли, гулять]",колька пошли гулять
21265,neutral,Ура! Золото дают бесплатно!Напиши это в 4 комм...,"[ура, золото, дают, бесплатно, напиши, это, в,...",ура золото дают бесплатно напиши это коммента...
21266,speech,"С Праздником, Ксюнь! Женского счастья тебе! Вы...","[с, праздником, ксюнь, женского, счастья, тебе...",праздником ксюнь женского счастья тебе выбира...


In [7]:
# Создание текстовых файлов для обучения fasttext модели с лейблом и текстом
def make_txt(data, column, file_name):
    with open(file_name, 'w') as f:
        for each_text, each_label in zip(data[column], data['label']):
            f.writelines(f'__label__{each_label} {each_text}\n')

# файлы с минимально обработаннымм текстами
make_txt(train, 'text', 'train-text.txt')
make_txt(test, 'text', 'test-text.txt')

print('Данные для обучения:')
! head -n 10 train-text.txt

Данные для обучения:
__label__negative А попа подозревала давно,что ты с кавказа..перестану общаться с тобой
__label__speech З прошедшим Днем Ангела))))))))
__label__skip Два дня до отлета с острова!!!!!!!
__label__negative Блин, почему эта жизнь столь не справедлива ((((
__label__skip где еще встречать свой день рождения как не на кладбище))))
__label__neutral Решите уравнение мужских дел для жизни : Домов - 3 Деревьев - 5 Сыновей - ?
__label__positive патамушта я крутая баба! ееее!!
__label__neutral Просто пост :)
__label__positive урря!я дождался этой овцыы)
__label__speech С Днем Рождения желаю много счастья, любви и успехов во всем:) И еще с днем Защитника Отечества;))))))))


In [8]:
# файлы с текстами без знаков препинания и стоп слов
make_txt(train, 'cleaned_text', 'train-cleaned.txt')
make_txt(test, 'cleaned_text', 'test-cleaned.txt')

print('Данные для обучения:')
! head -n 10 train-cleaned.txt

Данные для обучения:
__label__negative  попа подозревала давно кавказа перестану общаться тобой 
__label__speech  з прошедшим днем ангела 
__label__skip  дня отлета острова 
__label__negative  блин почему эта жизнь столь справедлива 
__label__skip  встречать свой день рождения кладбище 
__label__neutral  решите уравнение мужских дел жизни домов деревьев сыновей 
__label__positive  патамушта крутая баба ееее 
__label__neutral  просто пост 
__label__positive  урря дождался овцыы 
__label__speech  днем рождения желаю счастья любви успехов всем днем защитника отечества 


In [9]:
! pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import fasttext
from sklearn.metrics import f1_score

# Печать результатов обучения модели
def print_results(sample_size, precision, recall):
    precision = round(precision, 2)
    recall = round(recall, 2)
    print('Размер выборки = ', sample_size)
    print('Precision = ', precision)
    print('Recall = ', recall)

# Вычисление предсказаний для всех текстов и сравнение их с реальными
def preds_f1(model, test_column):
    pred = []
    for line in test_column:
        pred.append(model.predict(line)[0][0])

    f1 = f1_score('__label__'+test.label, pred, average='macro')
    print('\nf1_score = ', f1)

In [11]:
# Модель с базовыми параметрами на минимально обработанных данных
model0 = fasttext.train_supervised('train-text.txt')

print_results(*model0.test('test-text.txt'))
preds_f1(model0, test['text'])

Размер выборки =  2967
Precision =  0.64
Recall =  0.64

f1_score =  0.5301124813074349


In [12]:
# Модель с базовыми параметрами на данных без знаков препинания и стоп слов
model1 = fasttext.train_supervised('train-cleaned.txt')

print_results(*model1.test('test-cleaned.txt'))
preds_f1(model1, test['cleaned_text'])

Размер выборки =  2967
Precision =  0.67
Recall =  0.67

f1_score =  0.5918161895733179


Обучение для text оказалось лучше по train без знаков препинания и стоп слов

#### Теперь для той же задачи использую предобученную модель

In [13]:
! pip install dostoevsky
#! python dostoevsky download fasttext-social-network-model
! wget https://storage.b-labs.pro/models/fasttext-social-network-model.bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-10-23 13:40:01--  https://storage.b-labs.pro/models/fasttext-social-network-model.bin
Resolving storage.b-labs.pro (storage.b-labs.pro)... 104.21.14.222, 172.67.160.159, 2606:4700:3031::ac43:a09f, ...
Connecting to storage.b-labs.pro (storage.b-labs.pro)|104.21.14.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84051390 (80M) [application/x-sega-cd-rom]
Saving to: ‘fasttext-social-network-model.bin’

fasttext-social-net 100%[===================>]  80.16M   117MB/s    in 0.7s    

2022-10-23 13:40:01 (117 MB/s) - ‘fasttext-social-network-model.bin’ saved [84051390/84051390]



In [14]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
FastTextSocialNetworkModel.MODEL_PATH = './fasttext-social-network-model.bin'

tokenizer = RegexTokenizer()
model2 = FastTextSocialNetworkModel(tokenizer=tokenizer)


# Вывод первых двух по вероятности результатов для первых пяти строк теста
ex =  test['text'][0:5]
res = model2.predict(ex, k=2)

for message, sentiment in zip(ex, res):
    print(message, ' -> ', sentiment)

Александр, тебе к лицу эта пушка :)   ->  {'neutral': 0.30736804008483887, 'skip': 0.15611489117145538}
Скоро ты вернешься домой, грязный, не бритый но такой любимый ❤  ->  {'positive': 0.8596737384796143, 'neutral': 0.040855713188648224}
помниш...))  ->  {'positive': 0.46102678775787354, 'neutral': 0.4225146472454071}
Мы просто с [id12210212|Елисеем] психанули ))))) К новому году в бункере готовы!)))))) Да, мы любим молоко! Когда стареешь - переходишь на него вместо огненной воды🍹  ->  {'positive': 0.6442351341247559, 'neutral': 0.1259327530860901}
ой, прости забыла поздравить, с праздничьком, да ! ))))  ->  {'speech': 0.994098961353302, 'negative': 0.02518884465098381}


In [15]:
def dostoevsky_check(test, column):
    results = model2.predict(test[column], k=1)

    pred = []
    for sentiment in results:
        pred.append(sentiment.popitem()[0])
    
    f1 = f1_score(test.label, pred, average='macro')
    print('f1_score = ', f1)


print('train')
dostoevsky_check(train, 'text')
dostoevsky_check(train, 'cleaned_text')
print('test')
dostoevsky_check(test, 'text')
dostoevsky_check(test, 'cleaned_text')

train
f1_score =  0.8427868404031168
f1_score =  0.7429676199442827
test
f1_score =  0.6550226772849912
f1_score =  0.6048920995937775


Предобученная модель лучше себя показывает на начальном тексте, без удаления знаков препинания и стоп слов. 

Наилучший f1-score на тесте:

In [16]:
dostoevsky_check(test, 'text')

f1_score =  0.6550226772849912
